In [1]:
import os
import pandas as pd
import numpy as np
import scipy
import glob
import math
import matplotlib.pyplot as plt
from operator import *

import mne
from mne.preprocessing import ICA

from scipy.signal import butter, lfilter

import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

def butter_bandpass(lowcut, highcut, fs, order=6):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
def calcsnr(prefilter, filtered):
    filteredsum = 0
    denomsum = 0
    for i in range(len(prefilter)):
        filteredsum = filteredsum+filtered[i]*filtered[i]
        denomsum = denomsum+ (filtered[i]-prefilter[i])*(filtered[i]-prefilter[i])
    return 10*math.log10(filteredsum/denomsum)


class EEGDataset(Dataset):
    def __init__(self, eeglist, labels, transform=None, target_transform=None):
        self.labels = torch.from_numpy(np.array(labels))
        self.labels = self.labels.to(torch.float32)
        self.eeglist = eeglist
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        label = self.labels[idx]
        raw = self.eeglist[idx]
        eeg = torch.from_numpy(raw.get_data())
        eeg = eeg.to(torch.float32)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return eeg, label

data_list = {"raw":[], "preprocessed":[], "manual_preprocess":[], "rawsample": [], "preprocessedsample":[], "manual_preprocesssample":[]}
labels = []
readtsv = pd.read_csv('participants.tsv', sep = '\t')

channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
for i in range(1,89):

    filteredpath = r"D:\Laptop Archive\Sunny\School Work\UofT 2023-2024\APS360\derivatives\sub-%s\eeg"%(str(i).zfill(3))
    filteredfile = glob.glob(os.path.join(filteredpath, '*.set'))
    filtered = mne.io.read_raw_eeglab(filteredfile[0], preload = True)


    rawpath = r"D:\Laptop Archive\Sunny\School Work\UofT 2023-2024\APS360\Raw Data\sub-%s\eeg"%(str(i).zfill(3))
    file = glob.glob(os.path.join(rawpath, '*.set'))
    raw = mne.io.read_raw_eeglab(file[0], preload = True)

    data_list['raw'].append(raw)
    data_list['preprocessed'].append(filtered)
    

    label = readtsv['Group'][i-1]
    if label == 'A':
        labels.append([0,0,1])
    elif label == 'F':
        labels.append([0,1,0])
    elif label == 'C':
        labels.append([1,0,0])



C:\Users\sunny\AppData\Local\Temp\ipykernel_13284\2670626108.py:68: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  filtered = mne.io.read_raw_eeglab(filteredfile[0], preload = True)
C:\Users\sunny\AppData\Local\Temp\ipykernel_13284\2670626108.py:68: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  filtered = mne.io.read_raw_eeglab(filteredfile[0], preload = True)
C:\Users\sunny\AppData\Local\Temp\ipykernel_13284\2670626108.py:68: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  filtered = mne.io.read_raw_eeglab(filteredfile[0], preload = True)
C:\Users\sunny\AppData\Local\Temp\ipykernel_13284\2670626108.py:68: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. 

In [13]:
import os
import pandas as pd
import numpy as np
import scipy
import glob
import math
import matplotlib.pyplot as plt
from operator import *

import mne
from mne.preprocessing import ICA

from scipy.signal import butter, lfilter

import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from IPython.display import clear_output 


def butter_bandpass(lowcut, highcut, fs, order=6):
    return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
def calcsnr(prefilter, filtered):
    filteredsum = 0
    denomsum = 0
    for i in range(len(prefilter)):
        filteredsum = filteredsum+filtered[i]*filtered[i]
        denomsum = denomsum+ (filtered[i]-prefilter[i])*(filtered[i]-prefilter[i])
    return 10*math.log10(filteredsum/denomsum)


class EEGDataset(Dataset):
    def __init__(self, eeglist, labels, transform=None, target_transform=None):
        self.labels = torch.from_numpy(np.array(labels))
        self.labels = self.labels.to(torch.float32)
        self.eeglist = eeglist
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        label = self.labels[idx]
        raw = self.eeglist[idx]
        eeg = torch.from_numpy(raw.get_data())
        eeg = eeg.to(torch.float32)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return eeg, label

bandwidth_split_data = {'alpha': [], 'beta': [], 'delta': [], 'gamma': [],'alpha_processed': [], 'beta_processed': [], 'delta_processed': [], 'gamma_processed': []}
labels = []
readtsv = pd.read_csv('participants.tsv', sep = '\t')

def splitbandwidth(band, data):
    return data.filter(l_freq = band[0], h_freq=band[1])

channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
for i in range(1,89):

    rawpath = r"D:\Laptop Archive\Sunny\School Work\UofT 2023-2024\APS360\Raw Data\sub-%s\eeg"%(str(i).zfill(3))
    file = glob.glob(os.path.join(rawpath, '*.set'))
    raw = mne.io.read_raw_eeglab(file[0], preload = True)

    bandwidth_split_data['delta'].append(splitbandwidth([0.5, 4], raw.copy()))
    bandwidth_split_data['alpha'].append(splitbandwidth([8, 12], raw.copy()))
    bandwidth_split_data['beta'].append(splitbandwidth([13, 30], raw.copy()))
    bandwidth_split_data['gamma'].append(splitbandwidth([30, 80], raw.copy()))
    clear_output()

    label = readtsv['Group'][i-1]
    if label == 'A':
        labels.append([0,0,1])
    elif label == 'F':
        labels.append([0,1,0])
    elif label == 'C':
        labels.append([1,0,0])



In [15]:
from mne_icalabel import label_components
exclusion = ['line noise', 'heartbeat', 'eye blink']

def manual_process(raw, plotting = True):
    freq_low = 0.5
    freq_high = 45
    iirparams = dict(order = 4, ftype = 'butter')

    raw.filter(freq_low, freq_high, method = 'iir', iir_params = iirparams)

    # Create ICA object and fit it to the data
    ica = ICA(n_components=19, random_state=97, verbose = False)
    ica.fit(raw)

    # Plot ICA components to identify artifacts
    icalabels = label_components(raw, ica, method = 'iclabel')
    if plotting == True:
        ica.plot_components()
        picks = list(range(0,18))
        ica.plot_properties(raw, picks=picks)
        print(icalabels)
    ica.exclude = []
    for i, label in enumerate(icalabels):
        if label in exclusion:
            ica.exclude.append(i)
    ica.apply(raw)
            
    return raw

In [6]:
from IPython.display import clear_output 
for subject in data_list['raw']:
    raw = subject.copy().load_data()
    data_list['manual_preprocess'].append(manual_process(raw, plotting = False))
    clear_output()


In [16]:
from IPython.display import clear_output 
for n in range (len(bandwidth_split_data['alpha'])):
    alpha = bandwidth_split_data['alpha'][n].copy().load_data()
    beta = bandwidth_split_data['beta'][n].copy().load_data()
    gamma = bandwidth_split_data['gamma'][n].copy().load_data()
    delta = bandwidth_split_data['delta'][n].copy().load_data()
    bandwidth_split_data['alpha_processed'].append(manual_process(alpha, plotting = False))
    bandwidth_split_data['beta_processed'].append(manual_process(beta, plotting = False))
    bandwidth_split_data['gamma_processed'].append(manual_process(gamma, plotting = False))
    bandwidth_split_data['delta_processed'].append(manual_process(delta, plotting = False))
    clear_output()


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 45.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Selecting by number: 19 components
Fitting ICA took 9.8s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the

Applying ICA to Raw instance
    Transforming to ICA space (19 components)
    Zeroing out 0 ICA components
    Projecting back using 19 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 45.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Selecting by number: 19 components
Fitting ICA took 6.9s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the

Applying ICA to Raw instance
    Transforming to ICA space (19 components)
    Zeroing out 0 ICA components
    Projecting back using 19 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.50, 45.00 Hz: -6.02, -6.02 dB

Fitting ICA to data using 19 channels (please be patient, this may take a while)
Selecting by number: 19 components
Fitting ICA took 18.0s.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  icalabels = label_components(raw, ica, method = 'iclabel')
C:\Users\sunny\AppData\Local\Temp\ipykernel_32312\2725101412.py:16: RuntimeWarning: The provided ICA instance was fitted with a 'fastica' algorithm. ICLabel was designed with extended infomax ICA decompositions. To use the

In [8]:
import gc
gc.collect()
totals = [485.5, 276.5, 402]
trainlist, vallist, testlist, trainlabels, vallabels, testlabels = [],[],[],[],[],[]
labelonehot = [[0,0,1], [0,1,0], [1,0,0]]
#make sure enough of each group in each partition of data
for x, label in enumerate(labelonehot):
  count = 0
  for i in range (len(labels)):
    if labels[i] == label:
      m = 0
      # print(len(n.get_data()[0]))
      while 15000*(m+1) < len(data_list['manual_preprocess'][i].copy().get_data()[0]):
        if totals[x]*2-count>100:
          n = data_list['manual_preprocess'][i].copy()
          trainlist.append(n.crop(m*30,(m+1)*30))
          trainlabels.append(labels[i])
        if totals[x]*2-count>20:
          n = data_list['manual_preprocess'][i].copy()
          vallist.append(n.crop(m*30,(m+1)*30))
          vallabels.append(labels[i])
        else:
          n = data_list['manual_preprocess'][i].copy()
          testlist.append(n.crop(m*30, (m+1)*30))
          testlabels.append(labels[i])
        count = count + 1
        m = m+1


trainset = EEGDataset(trainlist, trainlabels)
valset = EEGDataset(vallist, vallabels)
testset = EEGDataset(testlist, testlabels)

trainloader = DataLoader(trainset, batch_size = 64, shuffle = True)
valloader = DataLoader(valset, batch_size = 64, shuffle = True)
testloader = DataLoader(testset, batch_size = 64, shuffle = True)


KeyboardInterrupt: 

In [ ]:
import gc
gc.collect()
totals = [485.5, 276.5, 402]
alphatrainlist, alphavallist, alphatestlist, alphatrainlabels, alphavallabels, alphatestlabels = [],[],[],[],[],[]
betatrainlist, betavallist, betatestlist, betatrainlabels, betavallabels, betatestlabels = [],[],[],[],[],[]
gammatrainlist, gammavallist, gammatestlist, gammatrainlabels, gammavallabels, gammatestlabels = [],[],[],[],[],[]
deltatrainlist, deltavallist, deltatestlist, deltatrainlabels, deltavallabels, deltatestlabels = [],[],[],[],[],[]
trainlist, vallist, testlist, trainlabels, vallabels, testlabels = [],[],[],[],[],[]

labelonehot = [[0,0,1], [0,1,0], [1,0,0]]
#make sure enough of each group in each partition of data
for x, label in enumerate(labelonehot):
  count = 0
  for i in range (len(labels)):
    if labels[i] == label:
      m = 0
      # print(len(n.get_data()[0]))
      while 15000*(m+1) < len(bandwidth_split_data['alpha'][i].copy().get_data()[0]):
        if totals[x]*2-count>100:
          A = bandwidth_split_data['alpha'][i].copy()
          B = bandwidth_split_data['beta'][i].copy()
          G = bandwidth_split_data['gamma'][i].copy()
          D = bandwidth_split_data['delta'][i].copy()
          alphatrainlist.append(A.crop(m*30,(m+1)*30))
          alphatrainlabels.append(labels[i])
          
          betatrainlist.append(B.crop(m*30,(m+1)*30))
          betatrainlabels.append(labels[i])
          gammatrainlist.append(G.crop(m*30,(m+1)*30))
          gammatrainlabels.append(labels[i])
          deltatrainlist.append(D.crop(m*30,(m+1)*30))
          deltatrainlabels.append(labels[i])
          trainlist.append(A)
          trainlist.append(B)
          trainlist.append(G)
          trainlist.append(D)
          for j in range(0,4):  
            trainlabels.append(labels[i])
        if totals[x]*2-count>20:
          A = bandwidth_split_data['alpha'][i].copy()
          B = bandwidth_split_data['beta'][i].copy()
          G = bandwidth_split_data['gamma'][i].copy()
          D = bandwidth_split_data['delta'][i].copy()
          alphavallist.append(A.crop(m*30,(m+1)*30))
          alphavallabels.append(labels[i])
          betavallist.append(B.crop(m*30,(m+1)*30))
          betavallabels.append(labels[i])
          gammavallist.append(G.crop(m*30,(m+1)*30))
          gammavallabels.append(labels[i])
          deltavallist.append(D.crop(m*30,(m+1)*30))
          deltavallabels.append(labels[i])
          vallist.append(A)
          vallist.append(B)
          vallist.append(G)
          vallist.append(D)
          for j in range(0,4):  
            vallabels.append(labels[i])
        else:
          A = bandwidth_split_data['alpha'][i].copy()
          B = bandwidth_split_data['beta'][i].copy()
          G = bandwidth_split_data['gamma'][i].copy()
          D = bandwidth_split_data['delta'][i].copy()
          alphatestlist.append(A.crop(m*30,(m+1)*30))
          alphatestlabels.append(labels[i])
          betatestlist.append(B.crop(m*30,(m+1)*30))
          betatestlabels.append(labels[i])
          gammatestlist.append(G.crop(m*30,(m+1)*30))
          gammatestlabels.append(labels[i])
          deltatestlist.append(D.crop(m*30,(m+1)*30))
          deltatestlabels.append(labels[i])
          testlist.append(A)
          testlist.append(B)
          testlist.append(G)
          testlist.append(D)
          for j in range(0,4):  
            testlabels.append(labels[i])
        count = count + 1
        m = m+1
        gc.collect()

alphatrainset = EEGDataset(alphatrainlist, alphatrainlabels)
betatrainset = EEGDataset(betatrainlist, betatrainlabels)
gammatrainset = EEGDataset(gammatrainlist, gammatrainlabels)
deltatrainset = EEGDataset(deltatrainlist, deltatrainlabels)
fulltrainset = EEGDataset(trainlist, trainlabels)

alphavalset = EEGDataset(alphavallist, alphavallabels)
betaavalset = EEGDataset(betavallist, betavallabels)
gammavalset = EEGDataset(gammavallist, gammavallabels)
deltavalset = EEGDataset(deltavallist, deltavallabels)
fullvalset = EEGDataset(vallist, vallabels)

alphatestset = EEGDataset(alphatestlist, alphatestlabels)
betaatestset = EEGDataset(betatestlist, betatestlabels)
gammatestset = EEGDataset(gammatestlist, gammatestlabels)
deltatestset = EEGDataset(deltatestlist, deltatestlabels)
fulltestset = EEGDataset(testlist, testlabels)

alphatrainloader = DataLoader(alphatrainset, batch_size = 64, shuffle = True)
betatrainloader = DataLoader(betatrainset, batch_size = 64, shuffle = True)
gammatrainloader = DataLoader(gammatrainset, batch_size = 64, shuffle = True)
deltatrainloader = DataLoader(deltatrainset, batch_size = 64, shuffle = True)

alphavalloader = DataLoader(alphavalset, batch_size = 64, shuffle = True)
betavalloader = DataLoader(betavalset, batch_size = 64, shuffle = True)
gammavalloader = DataLoader(gammavalset, batch_size = 64, shuffle = True)
deltavalloader = DataLoader(deltavalset, batch_size = 64, shuffle = True)

alphatestloader = DataLoader(alphatestset, batch_size = 64, shuffle = True)
betatestloader = DataLoader(betatestset, batch_size = 64, shuffle = True)
gammatestloader = DataLoader(gammatestset, batch_size = 64, shuffle = True)
deltatestloader = DataLoader(deltatestset, batch_size = 64, shuffle = True)

fulltrainloader = DataLoader(fulltrainset, batch_size = 64, shuffle = True)
fullvalloader = DataLoader(fullvalset, batch_size = 64, shuffle = True)
fulltestloader = DataLoader(fulltestset, batch_size = 64, shuffle = True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Define your ANN model
class ANN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(19*10001, 5000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5000, 300)
        self.fc3 = nn.Linear(300,3)

    def forward(self, x):
        x = x.view(-1, 19*10001)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x


# Define hyperparameters
hidden_size = 19
learning_rate = 0.001
num_epochs = 50
batch_size = 2

# Create DataLoader objects

# Initialize the ANN model
model = ANN(30001, hidden_size, 3)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss with Logits
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in trainloader:
        # Forward pass
        inputs = inputs.to(torch.float32)
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    # Print average loss for this epoch
    epoch_loss = running_loss / len(trainloader.dataset)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Evaluation
# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for imgs, labels in testloader:
        outputs = model(imgs)
        predicted = torch.argmax(outputs, dim=1)
        _, labels = torch.max(labels, dim=1)
        # print(predicted, labels)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = correct / total
    print(f'Accuracy on test set: {accuracy:.4f}')



In [15]:
import torch.nn as nn
import torch.nn.functional as F

class CNN_EEG_Classifier(nn.Module):
    def __init__(self):
        super(CNN_EEG_Classifier, self).__init__()
        #USE LARGER KERNEL BECAUSE IMAGES ARE HIGH RESOLUTION, FEATURES TAKE UP MORE PIXELS
        self.conv1 = nn.Conv1d(19, 25, 2) 
        
        #BATCH NORMALIZATION TO PREVENT VANISHING GRADIENTS
        self.bn1 = nn.BatchNorm1d(25)
        #LARGE POOLING KERNEL TO REDUCE DIMENSIONALIZATION FASTER (JUST FOUND THIS TO BE HELPFUL BY EXPERIMENTING)
        self.pool = nn.MaxPool1d(4, 4)

        self.conv2 = nn.Conv1d(25, 30, 2) 

        self.bn2 = nn.BatchNorm1d(30)
        self.fc1 = nn.Linear(28110, 256)
        self.fc2 = nn.Linear(256, 3)


    def forward(self, x):
        batch_size = x.shape[0]
        outconv1 = self.pool(F.relu(self.bn1(self.conv1(x))))
        outconv2 = self.pool(F.relu(self.bn2(self.conv2(outconv1))))

        outconv2 = outconv2.view(batch_size, -1)

        outfc1 = F.relu(self.fc1(outconv2))
        # print(outfc1.shape)

        outfc2 = self.fc2(outfc1)
        # print(outfc2.shape)

        out = F.softmax(outfc2, dim = 1)

        return out


In [16]:
import torch.optim as optim
import time

def get_accuracy(model, train=True, train_data = trainloader, val_data = valloader):
    if train:
        dataloader = train_data
    else:
        dataloader = val_data
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in dataloader:
            outputs = model(imgs)
            predicted = torch.argmax(outputs, dim=1)
            _, labels = torch.max(labels, dim=1)
            # print(predicted, labels)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    return 100 * correct / total
torch.manual_seed(5)
def train(model,batch_size=8, traindata = trainloader, valdata = valloader, num_epochs=10, rate = 0.001):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=rate, momentum=0.9)
    
    iters, losses, train_acc, val_acc = [], [], [], []

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(traindata):
            
            # print(imgs.shape)
            imgs = imgs.to(torch.float32)
            out = model(imgs)             # forward pass
            # print(out.shape)
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)

            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            train_acc.append(get_accuracy(model, train=True, train_data = traindata)) # compute training accuracy 
            val_acc.append(get_accuracy(model, train=False, val_data = valdata))  # compute validation accuracy
            n += 1
    print("Iterations:", n)
    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()
    
    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
eegclassifier = CNN_EEG_Classifier()
train(eegclassifier, batch_size = 64, num_epochs = 5)

: 

In [ ]:
print("Test accuracy:",get_accuracy(eegclassifier,train = True, train_data = testloader))